### 【 D0122_work_이준기 】

[26_01_22_과제]
- 알파벳을 사용하는 언어는 알파벳 빈도의 차이로 언어를 식별할 수 있습니다.
- 해당 데이터셋을 활용해서 언어 식별 모델을 생성하세요.
- 데이터셋
  * train 폴더 =>  나라영문2글자-숫자.txt
  * test 폴더  =>  나라영문2글자-숫자.txt

- 데이터셋 부족 시 Wikipedia 사이트에서 추가 가능 합니다

### 순서도 (계획)
1. txt파일 -> csv 파일로 변경하기 (문서마다의 알파벳 비율로 변경)
2. CSV파일을 불러와서 train, test 데이터프레임으로 저장 (슬라이싱을 통한 파일명 통일)
3. Feature과 Label을 분리하고, LabelEncoding을 통해서 숫자로 변경해주기
4. train과 test Tensor로 변경하고 모델 생성 (다중분류모델)
5. 완성된 모델 -> 정확도 계산을 통한 평가

In [39]:
## -------------------------------------------------------------
## txt 파일 -> 알파벳 비율 csv 파일로 변경하기
## -------------------------------------------------------------
import os                           # 폴더 안 파일 목록 가져오기 위한 운영체제
import csv                          # CSV 파일을 표 형식으로 저장
from collections import Counter     # 리스트 안 값의 빈도 계산 전용
import string                       # 

# 문서 하나를 입력받아 알파벳 26개의 비율을 반환하는 함수
def get_alphabet_ratios(text):
    alphabets = [char.lower() for char in text if char.isalpha()]   # 알파벳만 추출
    # 빈 문서 예외 처리
    if not alphabets:
        return {letter: 0 for letter in string.ascii_lowercase}
    
    # 알파벳 빈도 계산
    frequency = Counter(alphabets)
    total = len(alphabets)

    # 비율 계산
    return {letter: round(frequency.get(letter, 0) / total, 4) 
            for letter in string.ascii_lowercase}


# CSV 생성해주는 함수
def create_frequency_csv(folder_path, output_csv):
    rows = []   # 결과 저장용 리스트
    
    # 폴더 안에 있는 파일 순회하기
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder_path, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                text = f.read()
            
            ratios = get_alphabet_ratios(text)
            # 행 만들기
            row = {'filename': filename, **ratios} 
            rows.append(row)
    
    # CSV 파일 저장
    with open(output_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['filename'] + list(string.ascii_lowercase))
        writer.writeheader()
        writer.writerows(rows)
    
    print(f"생성 완료: {output_csv}")

# 실행
create_frequency_csv('./dataset/train/', 'train_frequency.csv')
create_frequency_csv('./dataset/test/', 'test_frequency.csv')


생성 완료: train_frequency.csv
생성 완료: test_frequency.csv


In [40]:
## -------------------------------------------------------------
## 사용해야 할 모듈 로딩
## -------------------------------------------------------------
import pandas as pd                                 # 데이터프레임 생성
from sklearn.preprocessing import LabelEncoder      # 파일명 라벨인코딩
import torch                                        # 텐서 및 수치, 기본 함수용 모듈
import torch.nn as nn                               # 인공신경망 관련 모듈
import torch.nn.functional as F                     # 인공신경망 함수 관련 모듈

from torchinfo import summary       # 모델 구조 확인용 유틸 모듈

In [41]:
## -------------------------------------------------------------
## CSV 파일 불러오기
## -------------------------------------------------------------
trainDF = pd.read_csv('train_frequency.csv')
testDF = pd.read_csv('test_frequency.csv')

In [42]:
## -------------------------------------------------------------
## 앞의 en, fr, id, tl만 사용할 거라서 파일명 슬라이싱을 통해 잘라주기
## -------------------------------------------------------------
trainDF['filename'] = trainDF['filename'].str[:2]
testDF['filename'] = testDF['filename'].str[:2]
trainDF.head()

,filename,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,en,0.0760,0.0128,0.0457,0.0461,0.1053,0.0157,0.0192,0.0437,0.0740,...,0.0000,0.0777,0.0614,0.0805,0.0259,0.0098,0.0141,0.0007,0.0200,0.0004
1,en,0.0840,0.0199,0.0303,0.0388,0.1367,0.0174,0.0312,0.0274,0.0752,...,0.0055,0.0899,0.0715,0.0776,0.0306,0.0137,0.0139,0.0020,0.0107,0.0006
2,en,0.0716,0.0122,0.0456,0.0326,0.1201,0.0147,0.0252,0.0235,0.0946,...,0.0017,0.0539,0.0880,0.0811,0.0290,0.0188,0.0119,0.0006,0.0180,0.0006
3,en,0.0720,0.0276,0.0299,0.0395,0.1207,0.0167,0.0235,0.0588,0.0650,...,0.0004,0.0590,0.0731,0.0934,0.0242,0.0051,0.0195,0.0060,0.0175,0.0017
4,en,0.0738,0.0204,0.0311,0.0396,0.1413,0.0204,0.0204,0.0569,0.0650,...,0.0004,0.0725,0.0596,0.0955,0.0250,0.0107,0.0239,0.0031,0.0149,0.0007


In [43]:
## -------------------------------------------------------------
## Feature과 Label 분리
## -------------------------------------------------------------
X_train = trainDF.loc[:, 'a':'z'].values
y_train = trainDF['filename'].values

X_test = testDF.loc[:, 'a':'z'].values
y_test = testDF['filename'].values

In [44]:
## -------------------------------------------------------------
## 라벨 인코딩을 통해 Label -> 숫자로 바꿔주기
## -------------------------------------------------------------
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)

In [45]:
## -------------------------------------------------------------
## Tensor로 변환
## -------------------------------------------------------------
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train_enc, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test_enc, dtype=torch.long)

In [46]:
## -------------------------------------------------------------
##          입력수         퍼셉트론수/출력수         AF
## -------------------------------------------------------------
## 입력층      26개               26개           ★ Pytorch에는
##                                               입력층 클래스 X
##                                               입력 텐서를 입력층으로 간주
## 은닉층      26개               64개           ReLU
## 은닉층      64개               32개           ReLU
## 출력층      32개                4개           -     분류
## -------------------------------------------------------------
## 클래스이름 : MNISTModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)   : 층 구성 요소 인스턴스 생성
##            forward(self, x) : 순전파 진행 메서드
##                               x -> 입력층으로 간주!
## -------------------------------------------------------------
class LangModel(nn.Module):
    ##- 층 구성 인스턴스 생성 메서드
    def __init__(self, num_classes):
        super().__init__()
        self.hd1_layer = nn.Linear(26, 64)
        self.hd2_layer = nn.Linear(64, 32)
        self.out_layer = nn.Linear(32, num_classes)

    ##- 순전파 진행 메서드
    def forward(self, x):
        ## 입력층 -> 은닉층 :  26 -> 64
        out = self.hd1_layer(x)
        out = F.relu(out)

        ## 은닉층 -> 은닉층 :  64 -> 32
        out = self.hd2_layer(out)
        out = F.relu(out)

        ## 은닉층 -> 은닉층 :  32 -> 4(num_classes의 개수)
        out = self.out_layer(out)

        return out


In [47]:
## -------------------------------------------------------------
## GPU 사용 가능 여부에 따라 연산 장치(Device) 자동 선택
## -------------------------------------------------------------
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'     # - CUDA 사용 가능 시 GPU(cuda), 그렇지 않으면 CPU 사용

In [48]:
## -------------------------------------------------------------
## 손실함수 & 최적화
## -------------------------------------------------------------
model = LangModel(num_classes=len(le.classes_)).to(DEVICE)   # 출력층 크기 선언

## -> 손실 계산 인스턴스
loss_fn = nn.CrossEntropyLoss()                   # 다중 분류 모델이므로 softmax 자동 적용

## -> 최적화 인스턴스
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [49]:
## -------------------------------------------------------------
## 모델 구조 확인
## -------------------------------------------------------------
summary(model, input_size=(1,26))

Layer (type:depth-idx)                   Output Shape              Param #
LangModel                                [1, 4]                    --
├─Linear: 1-1                            [1, 64]                   1,728
├─Linear: 1-2                            [1, 32]                   2,080
├─Linear: 1-3                            [1, 4]                    132
Total params: 3,940
Trainable params: 3,940
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.02

In [50]:
## -------------------------------------------------------------
## 모델과 데이터 간 device 불일치로 인한 RuntimeError 방지
## -------------------------------------------------------------
X_train = X_train.to(DEVICE)
y_train = y_train.to(DEVICE)

X_test = X_test.to(DEVICE)
y_test = y_test.to(DEVICE)

In [51]:
## -------------------------------------------------------------
## 학습 진행
## -------------------------------------------------------------
EPOCHS = 1000

for epoch in range(EPOCHS):
    optimizer.zero_grad()               # 1. 이전 기울기 제거를 통한 최적화
    logits = model(X_train)             # 2. 순전파
    loss = loss_fn(logits, y_train)     # 3. 손실 계산
    loss.backward()                     # 4. 역전파
    optimizer.step()                    # 5. 파라미터 업데이트


    ## 50에포크마다 손실 출력
    if epoch % 50 == 0:
        print(f"[{epoch+1:03}_에포크] loss : {loss.item():.6f}")


[001_에포크] loss : 1.388942
[051_에포크] loss : 1.352011
[101_에포크] loss : 1.110313
[151_에포크] loss : 0.793494
[201_에포크] loss : 0.642078
[251_에포크] loss : 0.516336
[301_에포크] loss : 0.383268
[351_에포크] loss : 0.245007
[401_에포크] loss : 0.149216
[451_에포크] loss : 0.094711
[501_에포크] loss : 0.063027
[551_에포크] loss : 0.043943
[601_에포크] loss : 0.031932
[651_에포크] loss : 0.024053
[701_에포크] loss : 0.018666
[751_에포크] loss : 0.014847
[801_에포크] loss : 0.012057
[851_에포크] loss : 0.009961
[901_에포크] loss : 0.008351
[951_에포크] loss : 0.007089


In [52]:
## -------------------------------------------------------------
## 정확도 계산을 통한 평가
## -------------------------------------------------------------
with torch.no_grad():                                       # torch.no_grad() : 역전파 계산 X, 기울기 계산 끄기
    pred_train = model(X_train).argmax(dim=1)               #                   (평가할 때는 학습 안 하므로 기울기 계산 끔)
    train_acc = (pred_train == y_train).float().mean()

    pred_test = model(X_test).argmax(dim=1)
    test_acc = (pred_test == y_test).float().mean()

## 정확도 출력
print(f"Train Acc: {train_acc:.4f}")
print(f"Test  Acc: {test_acc:.4f}")


Train Acc: 1.0000
Test  Acc: 1.0000


In [53]:
# 클래스 이름 (라벨 인코더 기준)
class_names = le.classes_   # ['en', 'fr', 'id', 'tl']

# -------------------------------------------------------------
# 예측 수행 (평가 모드)
# -------------------------------------------------------------
with torch.no_grad():
    logits = model(X_test)                     # 모델 출력 (logits)
    probs  = torch.softmax(logits, dim=1)      # 클래스별 확률
    preds  = probs.argmax(dim=1)               # 예측 클래스

# -------------------------------------------------------------
# 확률(%) + 예측/정답 정리 -> proba를 통해서 각 확률 계산해보기
# -------------------------------------------------------------
proba_df = pd.DataFrame(
    (probs.cpu().numpy() * 100).round(1),             
    columns=[f"{c}일 확률" for c in class_names]
)

proba_df["예측"] = [class_names[p] for p in preds.cpu().numpy()]   # 예측 라벨
proba_df["실제"] = [class_names[y] for y in y_test.cpu().numpy()]  # 실제 라벨
proba_df["정답여부"] = proba_df["예측"] == proba_df["실제"]         # 정답 여부

# 출력
proba_df

,en일 확률,fr일 확률,id일 확률,tl일 확률,예측,실제,정답여부
0,99.300003,0.100000,0.600000,0.000000,en,en,True
1,99.599998,0.200000,0.200000,0.000000,en,en,True
2,37.000000,63.000000,0.000000,0.000000,fr,fr,True
3,2.700000,97.300003,0.000000,0.000000,fr,fr,True
4,11.800000,0.000000,88.199997,0.000000,id,id,True
5,0.300000,0.000000,99.699997,0.000000,id,id,True
6,0.000000,0.000000,0.000000,100.000000,tl,tl,True
7,0.000000,0.000000,1.900000,98.099998,tl,tl,True


### 정리
- 실습 내용 : 텍스트 문서에서 추출한 알파벳 26개의 출현 비율을 입력 특징으로 사용하여 언어 분류 모델을 설계하고 학습
1. 입력층 26차원의 데이터를 두 개의 은닉층을 거쳐 4개의 언어 클래스(en, fr, id, tl)로 분류하도록 모델을 구성
2. 손실 함수로는 CrossEntropyLoss, 최적화 기법으로는 Adam을 사용

-> 학습 결과, 학습 데이터와 테스트 데이터 모두에서 높은 정확도를 보였으며, 특히 테스트 데이터에 대해서도 안정적인 분류 성능을 확인